# 4 2-D Anisotropic Diffusion

In this tutorial, we consider the following anisotropic diffusion problem

$$
	a(v, u) = \int_{\Omega}  \frac{\kappa_{\parallel}}{\|\mathbf{B}\|^2} (\textbf{B} \cdot \nabla v) (\textbf{B} \cdot \nabla u) + \kappa_{I} \nabla v \cdot \nabla u d\Omega, \quad \forall u,v \in \mathcal{V}_h
$$

where $\textbf{B}$ denotes the magnetic field, $\Omega$ is our 2D computational domain and $\mathcal{V}_h \subset H^1(\Omega)$. 
We are interested in highly anisotropic configurations with $\displaystyle \frac{\kappa_{\parallel}}{\kappa_{I}} \simeq 10^6 \gg 1$.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()